<a href="https://colab.research.google.com/github/dapsavoie/agricultural_satellite_classifier/blob/master/cnn_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pandas as pd 

from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/agri_vision/final_encoded.csv')

In [0]:
train_df.drop(columns='Unnamed: 0', inplace=True)
#strip train index
train_df['index'] = train_df['index'].str.rstrip('.png')
train_df['index'] = train_df['index'] + '.jpg'

columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.10) # set validation split

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df, 
    directory = '/content/drive/My Drive/agri_vision/Agriculture-Vision/train/images/rgb',
    x_col = 'index',
    y_col = columns,
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(256, 256),
    subset='training',
) # set as training data


validation_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df, 
    directory = '/content/drive/My Drive/agri_vision/Agriculture-Vision/train/images/rgb',
    x_col = 'index',
    y_col = columns,
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(256, 256),
    subset='validation',
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 1 invalid image filename(s) in x_col="index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 11610 validated image filenames.
Found 1290 validated image filenames.


In [0]:
input_shape = (256, 256, 3)

In [0]:
cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.2))

cnn1.add(Flatten())

cnn1.add(Dense(128, activation='relu'))
cnn1.add(Dense(6, activation='softmax'))

In [0]:
cnn1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
filepath="/content/drive/My Drive/agri_vision/Agriculture-Vision/cnn1.h5"

In [58]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

hist = cnn1.fit_generator(steps_per_epoch=50,generator=train_generator, validation_data= validation_generator, validation_steps=5,epochs=100, callbacks=[checkpoint,early], class_weight='auto')

Epoch 1/100
 1/50 [..............................] - ETA: 1:55 - loss: 5722.6934 - accuracy: 0.1250

KeyboardInterrupt: ignored